##Linear Regression to Estimate Delivery Duration for DoorDash

The dataset has about 197,429 rows and 16 features and contains a subset of deliveries received at DoorDash in early 2015 in a subset of the cities. Each row in this file corresponds to one unique delivery. The target value to predict here is the total seconds/minutes taken for delivery from when the order was created at and the delivery time.

##### Create a spark session and load the DoorDash Data set

In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import SparkSession

In [0]:
file_location = "/FileStore/tables/historical_data.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df.show()

+---------+-------------------+--------------------+--------+----------------------+--------------+-----------+--------+------------------+--------------+--------------+---------------------+------------------+------------------------+------------------------------+--------------------------------------------+
|market_id|         created_at|actual_delivery_time|store_id|store_primary_category|order_protocol|total_items|subtotal|num_distinct_items|min_item_price|max_item_price|total_onshift_dashers|total_busy_dashers|total_outstanding_orders|estimated_order_place_duration|estimated_store_to_consumer_driving_duration|
+---------+-------------------+--------------------+--------+----------------------+--------------+-----------+--------+------------------+--------------+--------------+---------------------+------------------+------------------------+------------------------------+--------------------------------------------+
|        1|2015-02-06 22:24:17| 2015-02-06 23:27:16|    1845|   

In [0]:
df.dtypes

Out[103]: [('market_id', 'string'),
 ('created_at', 'timestamp'),
 ('actual_delivery_time', 'string'),
 ('store_id', 'int'),
 ('store_primary_category', 'string'),
 ('order_protocol', 'string'),
 ('total_items', 'int'),
 ('subtotal', 'int'),
 ('num_distinct_items', 'int'),
 ('min_item_price', 'int'),
 ('max_item_price', 'int'),
 ('total_onshift_dashers', 'string'),
 ('total_busy_dashers', 'string'),
 ('total_outstanding_orders', 'string'),
 ('estimated_order_place_duration', 'int'),
 ('estimated_store_to_consumer_driving_duration', 'string')]

##### Data pre-processing

In [0]:
# Import the required libraries
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.sql.functions import *

In [0]:
# Create new timestamp column for all the attributes that had timestamp details stored as string
# Convert the string values to int
# Create the duration column (duration between order created time and actual delivery time in minutes and seconds)

df=df.withColumn('actual_delivery_time_ts',to_timestamp(df.actual_delivery_time)).\
                withColumn('market_id_int',df["market_id"].cast(IntegerType())).\
                withColumn('order_protocol_int',df["order_protocol"].cast(IntegerType())).\
                withColumn('total_onshift_dashers_int',df["total_onshift_dashers"].cast(IntegerType())).\
                withColumn('total_busy_dashers_int',df["total_busy_dashers"].cast(IntegerType())).\
                withColumn('total_outstanding_orders_int',df["total_outstanding_orders"].cast(IntegerType())).\
                withColumn('estimated_store_to_consumer_driving_duration_int',df["estimated_store_to_consumer_driving_duration"].cast(IntegerType())).\
                withColumn('estimated_order_place_minutes',round(col('estimated_order_place_duration')/60)).\
                withColumn('estimated_driving_duration_minutes',round(col('estimated_store_to_consumer_driving_duration_int')/60)).\
                withColumn('DurationInSeconds',col("actual_delivery_time_ts").cast("long")- col("created_at").cast("long")).\
                withColumn('DurationInMinutes',round(col('DurationInSeconds')/60))
                

In [0]:
#Dropping old columns

df=df.drop("market_id","actual_delivery_time","order_protocol","total_onshift_dashers","total_busy_dashers","total_outstanding_orders","estimated_store_to_consumer_driving_duration")
df.show()

+-------------------+--------+----------------------+-----------+--------+------------------+--------------+--------------+------------------------------+-----------------------+-------------+------------------+-------------------------+----------------------+----------------------------+------------------------------------------------+-----------------------------+----------------------------------+-----------------+-----------------+
|         created_at|store_id|store_primary_category|total_items|subtotal|num_distinct_items|min_item_price|max_item_price|estimated_order_place_duration|actual_delivery_time_ts|market_id_int|order_protocol_int|total_onshift_dashers_int|total_busy_dashers_int|total_outstanding_orders_int|estimated_store_to_consumer_driving_duration_int|estimated_order_place_minutes|estimated_driving_duration_minutes|DurationInSeconds|DurationInMinutes|
+-------------------+--------+----------------------+-----------+--------+------------------+--------------+------------

In [0]:
#create a new column to determine the time of the day the order was created at
df=df.withColumn('createdtime', date_format('created_at', 'HH:mm:ss'))

def timer(t):
    if t>= '6:00:00' and t<'12:00:00':
        return 'Morning'
    elif t>= '12:00:00' and t<'17:00:00':
        return 'Noon'
    elif t>= '17:00:00' and t<='23:59:59':
        return 'Evening'
    else:
        return 'Night'
    
func_udf = udf(timer)
df=df.withColumn("TimeOfDay",func_udf(df['createdtime']))

display(df)

created_at,store_id,store_primary_category,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,estimated_order_place_duration,actual_delivery_time_ts,market_id_int,order_protocol_int,total_onshift_dashers_int,total_busy_dashers_int,total_outstanding_orders_int,estimated_store_to_consumer_driving_duration_int,estimated_order_place_minutes,estimated_driving_duration_minutes,DurationInSeconds,DurationInMinutes,createdtime,TimeOfDay
2015-02-06T22:24:17.000+0000,1845,american,4,3441,4,557,1239,446,2015-02-06T23:27:16.000+0000,1,1,33,14,21,861,7.0,14.0,3779,63.0,22:24:17,Evening
2015-02-10T21:49:25.000+0000,5477,mexican,1,1900,1,1400,1400,446,2015-02-10T22:56:29.000+0000,2,2,1,2,2,690,7.0,12.0,4024,67.0,21:49:25,Evening
2015-01-22T20:39:28.000+0000,5477,NA,1,1900,1,1900,1900,446,2015-01-22T21:09:09.000+0000,3,1,1,0,0,690,7.0,12.0,1781,30.0,20:39:28,Evening
2015-02-03T21:21:45.000+0000,5477,NA,6,6900,5,600,1800,446,2015-02-03T22:13:00.000+0000,3,1,1,1,2,289,7.0,5.0,3075,51.0,21:21:45,Evening
2015-02-15T02:40:36.000+0000,5477,NA,3,3900,3,1100,1600,446,2015-02-15T03:20:26.000+0000,3,1,6,6,9,650,7.0,11.0,2390,40.0,02:40:36,Night
2015-01-28T20:30:38.000+0000,5477,NA,3,5000,3,1500,1900,446,2015-01-28T21:08:58.000+0000,3,1,2,2,2,338,7.0,6.0,2300,38.0,20:30:38,Evening
2015-01-31T02:16:36.000+0000,5477,NA,2,3900,2,1200,2700,446,2015-01-31T02:43:00.000+0000,3,1,10,9,9,638,7.0,11.0,1584,26.0,02:16:36,Night
2015-02-12T03:03:35.000+0000,5477,NA,4,4850,4,750,1800,446,2015-02-12T03:36:20.000+0000,3,1,7,8,7,626,7.0,10.0,1965,33.0,03:03:35,Night
2015-02-16T00:11:35.000+0000,5477,indian,4,4771,3,820,1604,446,2015-02-16T00:38:01.000+0000,2,3,8,6,18,289,7.0,5.0,1586,26.0,00:11:35,Night
2015-02-18T01:15:45.000+0000,5477,NA,2,2100,2,700,1200,446,2015-02-18T02:08:57.000+0000,3,1,2,2,2,715,7.0,12.0,3192,53.0,01:15:45,Night


In [0]:
#checking count of rows
df.count()

Out[108]: 197428

In [0]:
#dropping NA values
df=df.dropna()
df.count()

Out[109]: 179295

In [0]:
#create temporary table from the dataframe

temp_table_name = "temp" 
df.createOrReplaceTempView(temp_table_name)

In [0]:
#create permanent table from the dataframe to access it from all sessions

permanent_table_name = "Bt"
df.write.format("parquet").saveAsTable(permanent_table_name)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-3998948977833066> in <module>
      2 
      3 permanent_table_name = "pt"
----> 4 df.write.format("parquet").saveAsTable(permanent_table_name)

/databricks/spark/python/pyspark/sql/readwriter.py in saveAsTable(self, name, format, mode, partitionBy, **options)
    804         if format is not None:
    805             self.format(format)
--> 806         self._jwrite.saveAsTable(name)
    807 
    808     def json(self, path, mode=None, compression=None, dateFormat=None, timestampFormat=None,

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/

In [0]:
%sql
-- Check if table is created

select * from temp limit 5

created_at,store_id,store_primary_category,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,estimated_order_place_duration,actual_delivery_time_ts,market_id_int,order_protocol_int,total_onshift_dashers_int,total_busy_dashers_int,total_outstanding_orders_int,estimated_store_to_consumer_driving_duration_int,estimated_order_place_minutes,estimated_driving_duration_minutes,DurationInSeconds,DurationInMinutes,createdtime,TimeOfDay
2015-02-06T22:24:17.000+0000,1845,american,4,3441,4,557,1239,446,2015-02-06T23:27:16.000+0000,1,1,33,14,21,861,7.0,14.0,3779,63.0,22:24:17,Evening
2015-02-10T21:49:25.000+0000,5477,mexican,1,1900,1,1400,1400,446,2015-02-10T22:56:29.000+0000,2,2,1,2,2,690,7.0,12.0,4024,67.0,21:49:25,Evening
2015-01-22T20:39:28.000+0000,5477,NA,1,1900,1,1900,1900,446,2015-01-22T21:09:09.000+0000,3,1,1,0,0,690,7.0,12.0,1781,30.0,20:39:28,Evening
2015-02-03T21:21:45.000+0000,5477,NA,6,6900,5,600,1800,446,2015-02-03T22:13:00.000+0000,3,1,1,1,2,289,7.0,5.0,3075,51.0,21:21:45,Evening
2015-02-15T02:40:36.000+0000,5477,NA,3,3900,3,1100,1600,446,2015-02-15T03:20:26.000+0000,3,1,6,6,9,650,7.0,11.0,2390,40.0,02:40:36,Night


In [0]:
%sql
-- Check for outliers
select DurationInMinutes from temp order by 1 desc limit 10

DurationInMinutes
6231.0
5541.0
951.0
907.0
803.0
761.0
656.0
636.0
603.0
549.0


In [0]:
#removing outliers

df = df.filter(df.DurationInMinutes!='6231')
df = df.filter(df.DurationInMinutes!='5541')
df = df.filter(df.DurationInMinutes!='951')
df = df.filter(df.DurationInMinutes!='907')
df = df.filter(df.DurationInMinutes!='803')
df = df.filter(df.DurationInMinutes!='761')
df = df.filter(df.DurationInMinutes!='656')
df = df.filter(df.DurationInMinutes!='603')
df = df.filter(df.DurationInMinutes!='549')
df = df.filter(df.DurationInMinutes!='545')
df = df.filter(df.DurationInMinutes!='536')
df = df.filter(df.DurationInMinutes!='472')
df = df.filter(df.DurationInMinutes!='470')
df = df.filter(df.DurationInMinutes!='451')
df = df.filter(df.DurationInMinutes!='427')
df.count()

Out[114]: 179280

####Exploratory Data Analysis

In [0]:
#Summary statistics

df.select("DurationInMinutes","estimated_order_place_minutes","estimated_driving_duration_minutes").summary("mean","min","25%","50%","75%","max","stddev").show()

+-------+------------------+-----------------------------+----------------------------------+
|summary| DurationInMinutes|estimated_order_place_minutes|estimated_driving_duration_minutes|
+-------+------------------+-----------------------------+----------------------------------+
|   mean|47.684543730477465|            4.891253904506916|                 9.107083891120036|
|    min|               2.0|                          0.0|                               0.0|
|    25%|              35.0|                          4.0|                               6.0|
|    50%|              44.0|                          4.0|                               9.0|
|    75%|              56.0|                          7.0|                              12.0|
|    max|             636.0|                         45.0|                              35.0|
| stddev|18.722987223276135|           1.3938769112374645|                3.6550317126041736|
+-------+------------------+-----------------------------+--

In [0]:
%sql
-- Avg Duration in Minutes by Market

select market_id_int as Market_ID,round(avg(DurationInMinutes),1) as Avg_Duration_In_Minutes from temp group by 1 order by 2 desc

Market_ID,Avg_Duration_In_Minutes
1,51.5
6,47.8
3,47.5
4,47.4
5,46.5
2,46.1


Output can only be rendered in Databricks

In [0]:
%sql
-- Avg Duration in Minutes by Order Protocol

select order_protocol_int as Order_Protocol,round(avg(DurationInMinutes),1) as Avg_Duration_In_Minutes from temp group by 1 order by 2 desc

Order_Protocol,Avg_Duration_In_Minutes
6,63.3
1,50.0
4,48.2
2,47.4
3,47.1
5,45.7
7,41.3


Output can only be rendered in Databricks

In [0]:
%sql
-- Number of Orders and Avg Duration in Minutes by Time of the Day

select TimeOfDay,count(*) as Total_Number_of_Orders,round(avg(DurationInMinutes),1) as Avg_Duration_In_Minutes from temp group by 1
order by 2 desc

TimeOfDay,Total_Number_of_Orders,Avg_Duration_In_Minutes
Night,116661,50.1
Evening,60108,43.3
Noon,2526,48.7


Output can only be rendered in Databricks

###Building the Linear Regression Model

In [0]:
# Import the required libraries

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler,StringIndexer,StandardScaler
from pyspark.ml import Pipeline

In [0]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to estimate duration

data=df.select(['store_id','store_primary_category','subtotal','num_distinct_items','market_id_int','estimated_order_place_duration','total_busy_dashers_int',
                                 'order_protocol_int','total_onshift_dashers_int','estimated_store_to_consumer_driving_duration_int','TimeOfDay','DurationInMinutes'])

In [0]:
# Create a 70-30 train test split

train_data,test_data=data.randomSplit([0.7,0.3])

In [0]:
data.count()

Out[122]: 179280

In [0]:
# Use StringIndexer to convert the categorical columns to hold numerical data


store_category_indexer = StringIndexer(inputCol='store_primary_category',outputCol='store_category_index',handleInvalid='keep')
timeofday_indexer = StringIndexer(inputCol='TimeOfDay',outputCol='timeofday_index',handleInvalid='keep')



In [0]:
# Vector assembler is used to create a vector of input features

features = ["store_id","store_category_index","timeofday_index","num_distinct_items","market_id_int","subtotal","estimated_order_place_duration","total_onshift_dashers_int","estimated_store_to_consumer_driving_duration_int","order_protocol_int"]

assembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")

In [0]:
# StandardScaler is used to resize the distribution of values
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

In [0]:
# Pipeline is used to pass the data through indexer,assembler and StandardScalar simultaneously. Also, it helps to pre-rocess the test data
# in the same way as that of the train data
pipe = Pipeline(stages=[store_category_indexer,timeofday_indexer, assembler, standardScaler])

In [0]:
fitted_pipe=pipe.fit(train_data)

In [0]:
train_data=fitted_pipe.transform(train_data)
train_data.show()


+--------+----------------------+--------+------------------+-------------+------------------------------+----------------------+------------------+-------------------------+------------------------------------------------+---------+-----------------+--------------------+---------------+--------------------+--------------------+
|store_id|store_primary_category|subtotal|num_distinct_items|market_id_int|estimated_order_place_duration|total_busy_dashers_int|order_protocol_int|total_onshift_dashers_int|estimated_store_to_consumer_driving_duration_int|TimeOfDay|DurationInMinutes|store_category_index|timeofday_index|   unscaled_features|            features|
+--------+----------------------+--------+------------------+-------------+------------------------------+----------------------+------------------+-------------------------+------------------------------------------------+---------+-----------------+--------------------+---------------+--------------------+--------------------+
|      

In [0]:
# Create an object for the Linear Regression model

lr_model = LinearRegression(labelCol='DurationInMinutes')

In [0]:
# Fit the model on the train data

fit_model = lr_model.fit(train_data.select(['features','DurationInMinutes']))

In [0]:
# Transform the test data using the model to predict the duration

test_data=fitted_pipe.transform(test_data)
test_data.show()

+--------+----------------------+--------+------------------+-------------+------------------------------+----------------------+------------------+-------------------------+------------------------------------------------+---------+-----------------+--------------------+---------------+--------------------+--------------------+
|store_id|store_primary_category|subtotal|num_distinct_items|market_id_int|estimated_order_place_duration|total_busy_dashers_int|order_protocol_int|total_onshift_dashers_int|estimated_store_to_consumer_driving_duration_int|TimeOfDay|DurationInMinutes|store_category_index|timeofday_index|   unscaled_features|            features|
+--------+----------------------+--------+------------------+-------------+------------------------------+----------------------+------------------+-------------------------+------------------------------------------------+---------+-----------------+--------------------+---------------+--------------------+--------------------+
|      

In [0]:
# Store the results in a dataframe

results = fit_model.transform(test_data)
results.show()

+--------+----------------------+--------+------------------+-------------+------------------------------+----------------------+------------------+-------------------------+------------------------------------------------+---------+-----------------+--------------------+---------------+--------------------+--------------------+------------------+
|store_id|store_primary_category|subtotal|num_distinct_items|market_id_int|estimated_order_place_duration|total_busy_dashers_int|order_protocol_int|total_onshift_dashers_int|estimated_store_to_consumer_driving_duration_int|TimeOfDay|DurationInMinutes|store_category_index|timeofday_index|   unscaled_features|            features|        prediction|
+--------+----------------------+--------+------------------+-------------+------------------------------+----------------------+------------------+-------------------------+------------------------------------------------+---------+-----------------+--------------------+---------------+------------

In [0]:
results.select(['DurationInMinutes','prediction']).show()

+-----------------+------------------+
|DurationInMinutes|        prediction|
+-----------------+------------------+
|             33.0|41.008776939140176|
|             48.0| 50.09107691915716|
|             30.0|36.747712406750054|
|             17.0| 33.20470338640924|
|             63.0|49.254652608213576|
|             23.0| 38.02207595908785|
|             27.0| 41.10242676080142|
|             37.0| 45.74830350105151|
|             39.0|50.268247104832405|
|             23.0|43.610270989584414|
|             33.0|45.077873178901875|
|            102.0| 51.98544218032387|
|             46.0|  52.0622392863273|
|             63.0| 52.44205790680889|
|             88.0| 59.11886622251916|
|             42.0| 52.32744160293766|
|             55.0|56.628942861190474|
|             30.0| 43.95563594535295|
|             21.0|38.130010593810255|
|             22.0| 30.94645356428725|
+-----------------+------------------+
only showing top 20 rows



###Evaluating the model

In [0]:
test_results = fit_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+-------------------+
|          residuals|
+-------------------+
| -8.008776939140176|
|-2.0910769191571603|
| -6.747712406750054|
|-16.204703386409243|
| 13.745347391786424|
| -15.02207595908785|
|-14.102426760801421|
|  -8.74830350105151|
|-11.268247104832405|
|-20.610270989584414|
|-12.077873178901875|
|  50.01455781967613|
| -6.062239286327298|
| 10.557942093191109|
|  28.88113377748084|
|-10.327441602937661|
| -1.628942861190474|
|-13.955635945352952|
|-17.130010593810255|
|  -8.94645356428725|
+-------------------+
only showing top 20 rows



In [0]:
test_results.rootMeanSquaredError

Out[76]: 17.489280574124898

In [0]:
test_results.r2

Out[77]: 0.12024876487045666

######R2 value of 12.0% indicates that the model explains only about 12.0% variance in the Delivery Duration

In [0]:
 # Finding Beta Coefficients
fit_model.coefficients

Out[78]: DenseVector([-0.1091, 0.0249, -1.9676, 0.5983, -1.0626, 2.9516, 2.0348, 0.3298, 4.2695, 0.2093])

####Interpretation

In [0]:

As the number of distinct items increases by 1 count, the delivery duration increases by 0.5983 minutes
As the subtotal amount increases by 1 dollar, the delivery duration increases by 2.9516 minutes
As the estimated order place duration increases by 1 minute, the delivery duration increases by 2.0348 minutes
As the total onshift dashers increases by 1 person, the delivery duration increases by 0.3298 minutes
As the estimated store to consumer driving duration increases by 1 minute, the delivery duration increases by 4.2695 minutes.

The independent factors chosen for the model explain about 12% variance in the Delivery Duration. Although the model explains only about 12.0% variance, DoorDash can use this model to reasonably predict the delivery duration based on the above factors.

The average delivery duration is around 47 minutes.
The number of orders placed and average duration is highest for orders placed at night, compared to noon and evening. Hence, we can have more on-shift dashers during night to provide better customer service.
The average duration is highest for orders placed through protocol 6. Hence, we can have work on order protocol 6 to reduce average delivery duration.
The average duration is very higher for market ID 1. Hence, we can have more on-shift dashers in this region to provide better customer service.

